In [ ]:
# === ONE-CELL: build prompts for ONLY your generated images ====================
# >>>>>> EDIT THESE (use ABSOLUTE paths) <<<<<<
COCO_JSON    = "/abs/path/to/scripts/utils/captions_val2014.json"
REAL_DIR     = "/home/group-3/tests_L/MixDQ/scripts/utils/captions_val2014.json"                      # reference images
FP16_DIR     = "/home/group-3/tests_L/MixDQ/logs/sdxl_fp_eval_big/generated_images"
MIXDQ_DIR    = "/home/group-3/tests_L/MixDQ/logs/sdxl_mixdq_eval_images_big"
OUT_FP16_TXT = "/home/group-3/tests_L/MixDQ/scripts/utils/prompts_fp16.txt"
OUT_MIXD_TXT = "/home/group-3/tests_L/MixDQ/scripts/utils/prompts_mixdq.txt"
OUT_FP16_CSV = "/home/group-3/tests_L/MixDQ/scripts/utils/prompts_fp16.csv"
OUT_MIXD_CSV = "/home/group-3/tests_L/MixDQ/scripts/utils/prompts_mixdq.csv"

# ------------------------------------------------------------------------------
import os, re, json, csv, glob, pathlib
from typing import List, Tuple, Dict

def _abs(p): return os.path.abspath(os.path.expanduser(p))
def list_images_recursive(root, exts=(".png",".jpg",".jpeg",".bmp",".webp")) -> List[str]:
    root = _abs(root)
    return [p for p in glob.iglob(os.path.join(root, "**", "*"), recursive=True)
            if os.path.isfile(p) and p.lower().endswith(exts)]

def natural_key(s: str):
    import re
    b = os.path.basename(s)
    return [int(t) if t.isdigit() else t.lower() for t in re.split(r"(\d+)", b)]

def load_coco_maps(coco_json: str):
    with open(_abs(coco_json), "r", encoding="utf-8") as f:
        coco = json.load(f)
    id2fname = {im["id"]: im["file_name"] for im in coco["images"]}
    # use the FIRST caption per image_id for determinism (you can change this)
    id2cap: Dict[int, str] = {}
    for ann in coco["annotations"]:
        if ann["image_id"] not in id2cap:
            id2cap[ann["image_id"]] = ann["caption"].strip()
    return id2fname, id2cap

def find_coco_id_from_name(name: str, id2fname: Dict[int, str]):
    # try to extract a 12-digit COCO id anywhere in the filename
    m = re.search(r"(?<!\d)(\d{12})(?!\d)", name)
    if m:
        iid = int(m.group(1))
        if iid in id2fname:
            return iid, "12digit"
    return None, None

def build_prompts_for_folder(gen_dir: str, id2fname: Dict[int,str], id2cap: Dict[int,str],
                             out_txt: str, out_csv: str):
    files = sorted(list_images_recursive(gen_dir), key=natural_key)
    matched_prompts: List[str] = []
    rows = []
    n_matched = 0
    for p in files:
        base = os.path.basename(p)
        iid, how = find_coco_id_from_name(base, id2fname)
        if iid is not None and iid in id2cap:
            cap = id2cap[iid]
            matched_prompts.append(cap)
            rows.append({"filename": base, "image_id": iid, "coco_file_name": id2fname[iid],
                         "prompt": cap, "match_method": how})
            n_matched += 1
        else:
            # unmatched -> leave placeholder so line count still equals #images
            cap = "a photo"
            matched_prompts.append(cap)
            rows.append({"filename": base, "image_id": "", "coco_file_name": "",
                         "prompt": cap, "match_method": "UNMATCHED"})
    # write outputs
    with open(_abs(out_txt), "w", encoding="utf-8") as f:
        f.write("\n".join(matched_prompts))
    with open(_abs(out_csv), "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=["filename","image_id","coco_file_name","prompt","match_method"])
        w.writeheader(); w.writerows(rows)
    print(f"{gen_dir}: {n_matched}/{len(files)} matched to COCO captions")
    print(f"  -> prompts: {out_txt}")
    print(f"  -> mapping: {out_csv}")
    if n_matched < len(files):
        print("  NOTE: Some files didn’t contain a 12-digit COCO id; see mapping CSV (match_method=UNMATCHED).")

# ---- run ---------------------------------------------------------------------
id2fname, id2cap = load_coco_maps(COCO_JSON)
build_prompts_for_folder(GEN_DIR_FP16, id2fname, id2cap, OUT_FP16_TXT, OUT_FP16_CSV)
build_prompts_for_folder(GEN_DIR_MIXD, id2fname, id2cap, OUT_MIXD_TXT, OUT_MIXD_CSV)
# ==============================================================================


In [1]:
# === ONE-CELL: build prompts for your 1024 imgs + FID + CLIPScore + RI@1 ======
# >>>>>> EDIT THESE (absolute paths) <<<<<<
COCO_JSON    = "/abs/path/to/scripts/utils/captions_val2014.json"
REAL_DIR     = "/home/group-3/tests_L/MixDQ/scripts/utils/captions_val2014.json"                      # reference images
FP16_DIR     = "/home/group-3/tests_L/MixDQ/logs/sdxl_fp_eval_big/generated_images"
MIXDQ_DIR    = "/home/group-3/tests_L/MixDQ/logs/sdxl_mixdq_eval_images_big"
OUT_FP16_TXT = "/home/group-3/tests_L/MixDQ/scripts/utils/prompts_fp16.txt"
OUT_MIXD_TXT = "/home/group-3/tests_L/MixDQ/scripts/utils/prompts_mixdq.txt"
OUT_FP16_CSV = "/home/group-3/tests_L/MixDQ/scripts/utils/prompts_fp16.csv"
OUT_MIXD_CSV = "/home/group-3/tests_L/MixDQ/scripts/utils/prompts_mixdq.csv"

# ------------------------------------------------------------------------------
import os, re, json, glob, csv, tempfile, shutil, pathlib
from typing import List, Dict, Tuple

import torch, torch.nn.functional as F
from PIL import Image
from tqdm import tqdm

# Clean-FID
from cleanfid import fid

# Try open_clip; fallback to HuggingFace transformers CLIP
class ClipBackend:
    def __init__(self, device="cuda" if torch.cuda.is_available() else "cpu"):
        self.device = device
        self.name = None
        try:
            import open_clip
            self.oc = open_clip
            self.model, _, self.preprocess = open_clip.create_model_and_transforms(
                "ViT-L-14-336", pretrained="openai", device=device
            )
            self.tokenizer = open_clip.get_tokenizer("ViT-L-14-336")
            self.model.eval()
            self.name = "open_clip"
        except Exception:
            from transformers import CLIPModel, CLIPProcessor
            self.hf_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14-336").to(device)
            self.hf_proc  = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14-336")
            self.hf_model.eval()
            self.name = "hf_clip"

    @torch.no_grad()
    def embed_images(self, files: List[str], batch=64) -> torch.Tensor:
        feats = []
        if self.name == "open_clip":
            for i in tqdm(range(0, len(files), batch), desc="embed images (open_clip)"):
                imgs = []
                for p in files[i:i+batch]:
                    try: imgs.append(self.preprocess(Image.open(p).convert("RGB")))
                    except: continue
                if not imgs: continue
                ims = torch.stack(imgs).to(self.device)
                f = self.model.encode_image(ims)
                feats.append(F.normalize(f.float(), dim=-1))
        else:
            for i in tqdm(range(0, len(files), batch), desc="embed images (hf_clip)"):
                imgs = []
                for p in files[i:i+batch]:
                    try: imgs.append(Image.open(p).convert("RGB"))
                    except: continue
                if not imgs: continue
                inputs = self.hf_proc(images=imgs, return_tensors="pt").to(self.device)
                f = self.hf_model.get_image_features(**inputs)
                feats.append(F.normalize(f.float(), dim=-1))
        if not feats:
            raise ValueError("No image features computed.")
        return torch.cat(feats, dim=0)

    @torch.no_grad()
    def embed_texts(self, texts: List[str], batch=256) -> torch.Tensor:
        feats = []
        if self.name == "open_clip":
            tok = self.tokenizer
            for i in tqdm(range(0, len(texts), batch), desc="embed texts (open_clip)"):
                t = tok(texts[i:i+batch]).to(self.device)
                f = self.model.encode_text(t)
                feats.append(F.normalize(f.float(), dim=-1))
        else:
            from transformers import CLIPProcessor  # already imported in __init__
            for i in tqdm(range(0, len(texts), batch), desc="embed texts (hf_clip)"):
                inputs = self.hf_proc(text=texts[i:i+batch], return_tensors="pt", padding=True, truncation=True).to(self.device)
                f = self.hf_model.get_text_features(**inputs)
                feats.append(F.normalize(f.float(), dim=-1))
        return torch.cat(feats, dim=0)

# --------- general helpers -----------------------------------------------------
def _abs(p): return os.path.abspath(os.path.expanduser(p))
def list_images_recursive(root, exts=(".png",".jpg",".jpeg",".bmp",".webp")) -> List[str]:
    root = _abs(root)
    return [p for p in glob.iglob(os.path.join(root, "**", "*"), recursive=True)
            if os.path.isfile(p) and p.lower().endswith(exts)]

def natural_key(path: str):
    import re
    b = os.path.basename(path)
    return [int(t) if t.isdigit() else t.lower() for t in re.split(r"(\d+)", b)]

def flatten_to_tmp(files: List[str], tag: str) -> str:
    out = pathlib.Path(tempfile.mkdtemp(prefix=f"fid_flat_{tag}_")).resolve()
    for i, src in enumerate(files):
        ext = pathlib.Path(src).suffix.lower() or ".png"
        dst = out / f"{i:08d}{ext}"
        try: os.symlink(_abs(src), dst)
        except Exception: shutil.copy2(_abs(src), dst)
    return str(out)

# --------- COCO caption mapping + per-folder prompts --------------------------
def load_coco_maps(coco_json: str) -> Tuple[Dict[int,str], Dict[int,str]]:
    with open(_abs(coco_json), "r", encoding="utf-8") as f:
        coco = json.load(f)
    id2fname = {im["id"]: im["file_name"] for im in coco["images"]}
    id2cap: Dict[int,str] = {}
    for ann in coco["annotations"]:
        id2cap.setdefault(ann["image_id"], ann["caption"].strip())  # first caption per image
    return id2fname, id2cap

def extract_coco_id_from_name(name: str):
    m = re.search(r"(?<!\d)(\d{12})(?!\d)", name)
    return int(m.group(1)) if m else None

def build_prompts_for_folder(gen_dir: str, id2fname: Dict[int,str], id2cap: Dict[int,str],
                             out_txt: str, out_csv: str) -> List[str]:
    files = sorted(list_images_recursive(gen_dir), key=natural_key)
    prompts, rows = [], []
    matched = 0
    for p in files:
        base = os.path.basename(p)
        iid = extract_coco_id_from_name(base)
        if iid is not None and iid in id2cap:
            cap = id2cap[iid]
            matched += 1
            rows.append({"filename": base, "image_id": iid, "coco_file_name": id2fname.get(iid,""),
                         "prompt": cap, "match_method": "12digit"})
        else:
            cap = "a photo"  # fallback placeholder
            rows.append({"filename": base, "image_id": "", "coco_file_name": "",
                         "prompt": cap, "match_method": "UNMATCHED"})
        prompts.append(cap)
    with open(_abs(out_txt), "w", encoding="utf-8") as f: f.write("\n".join(prompts))
    with open(_abs(out_csv), "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=["filename","image_id","coco_file_name","prompt","match_method"])
        w.writeheader(); w.writerows(rows)
    print(f"{gen_dir}: matched {matched}/{len(files)} to COCO captions -> {out_txt} (and {out_csv})")
    return prompts, files

# --------- Metrics -------------------------------------------------------------
def clip_and_ri1(files: List[str], prompts: List[str], backend: ClipBackend) -> Tuple[float, float]:
    files_sorted = [f for _, f in sorted(zip([natural_key(x) for x in files], files))]
    if len(files_sorted) != len(prompts):
        raise ValueError(f"Images ({len(files_sorted)}) vs prompts ({len(prompts)}) mismatch.")
    img_feat = backend.embed_images(files_sorted)
    txt_feat = backend.embed_texts(prompts)
    S = txt_feat @ img_feat.T
    diag = torch.diag(S)
    clip_mean = (torch.clamp(diag, min=0.0) * 2.5).mean().item()
    top1 = S.argmax(dim=1)
    ri1 = (top1 == torch.arange(S.size(0), device=top1.device)).float().mean().item()
    return clip_mean, ri1

def fid_folder_vs_folder(real_files: List[str], gen_files: List[str], tag: str) -> float:
    flat_real = flatten_to_tmp(real_files, f"real_{tag}")
    flat_gen  = flatten_to_tmp(gen_files,  f"gen_{tag}")
    score = fid.compute_fid(flat_real, flat_gen, mode="clean", num_workers=0, verbose=True)
    print(f"[{tag}] temp real dir: {flat_real}\n[{tag}] temp gen  dir: {flat_gen}")
    return score

# --------- RUN ----------------------------------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# Gather image lists
real_files = sorted(list_images_recursive(REAL_DIR), key=natural_key)
fp16_files = sorted(list_images_recursive(FP16_DIR), key=natural_key)
mixd_files = sorted(list_images_recursive(MIXDQ_DIR), key=natural_key)
print(f"Found files  real={len(real_files)} | fp16={len(fp16_files)} | mixdq={len(mixd_files)}")
if len(real_files) == 0 or len(fp16_files) == 0 or len(mixd_files) == 0:
    raise FileNotFoundError("One of the folders has 0 readable images. Use ABSOLUTE paths; nested trees are OK.")

# Build per-folder prompts
id2fname, id2cap = load_coco_maps(COCO_JSON)
prompts_fp16, fp16_files = build_prompts_for_folder(FP16_DIR, id2fname, id2cap, OUT_FP16_TXT, OUT_FP16_CSV)
prompts_mixd, mixd_files = build_prompts_for_folder(MIXDQ_DIR, id2fname, id2cap, OUT_MIXD_TXT, OUT_MIXD_CSV)

# CLIP + RI@1
backend = ClipBackend(device=device)
print("CLIP backend:", backend.name)
clip_fp16, ri1_fp16 = clip_and_ri1(fp16_files, prompts_fp16, backend)
clip_mixd, ri1_mixd = clip_and_ri1(mixd_files, prompts_mixd, backend)

# FID
fid_fp16  = fid_folder_vs_folder(real_files, fp16_files, tag="FP16")
fid_mixdq = fid_folder_vs_folder(real_files, mixd_files,  tag="MixDQ")

# Summary
print("\n=== SUMMARY (1024 subset) ===")
print(f"CLIPScore (mean)  FP16:  {clip_fp16:.4f}    MixDQ: {clip_mixd:.4f}")
print(f"RI@1              FP16:  {ri1_fp16*100:5.2f}%  MixDQ: {ri1_mixd*100:5.2f}%")
print(f"FID (clean)       FP16:  {fid_fp16:.4f}       MixDQ: {fid_mixdq:.4f}")
# ==============================================================================


Device: cuda
Found files  real=0 | fp16=1024 | mixdq=1024


FileNotFoundError: One of the folders has 0 readable images. Use ABSOLUTE paths; nested trees are OK.